In [1]:
import numpy as np
from tqdm import tqdm
import xarray as xr
import matplotlib.pyplot as plt
import warnings
from itertools import product
from polarization_controller import polarization_bidi
from scipy.optimize import minimize
warnings.filterwarnings("ignore")
plt.style.use("plot_style.mplstyle")

In [2]:
p_bidi = polarization_bidi(tx_num_mzi_stages = 1, psr_bool=True, mzi_bool=True, rx_num_mzi_stages=1)

In [3]:
connector_return_loss_range = np.array([-30, -40, -50, -60])
mc_samples = 501
no_fibers = 7
phase_shifts = np.linspace(0, 2*np.pi, mc_samples * no_fibers)
rotation_rx = phase_shifts[np.random.permutation(mc_samples * no_fibers)]
rotation_ry = phase_shifts[np.random.permutation(mc_samples * no_fibers)]
rotation_rz = phase_shifts[np.random.permutation(mc_samples * no_fibers)]
rotation_delta = phase_shifts[np.random.permutation(mc_samples * no_fibers)]

In [4]:
params = []

In [5]:
def objective_function(voltages):
  """
  Objective function for the optimization.
  Args:
      bidi: Bidi class representing the optical link.
      voltages: Voltages applied to the phase shifters.
      input_state: Input state of the device.
      params: Dictionary of parameters for the optimization.
  Returns:
      The cost function.
  """

  input_state_forward = np.array([1, 0, 0, 0])

  tx_pa_voltage = voltages[0]
  tx_mzi_voltage = voltages[1]
  rx_pa_voltage = voltages[2]
  rx_mzi_voltage = voltages[3]

  p_bidi.bidi_tx.pa.XPS1.heater_voltage = 0.0
  p_bidi.bidi_tx.pa.XPS2.heater_voltage = 0.0
  if tx_pa_voltage > 0.0:
    p_bidi.bidi_tx.pa.XPS1.heater_voltage = tx_pa_voltage
  if tx_pa_voltage < 0.0:
    p_bidi.bidi_tx.pa.XPS2.heater_voltage = -tx_pa_voltage

  p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = 0.0
  p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = 0.0
  if tx_mzi_voltage > 0.0:
    p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = tx_mzi_voltage
  if tx_mzi_voltage < 0.0:
    p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = -tx_mzi_voltage

  p_bidi.bidi_rx.pa.XPS1.heater_voltage = 0.0
  p_bidi.bidi_rx.pa.XPS2.heater_voltage = 0.0
  if rx_pa_voltage > 0.0:
    p_bidi.bidi_rx.pa.XPS1.heater_voltage = rx_pa_voltage
  if rx_pa_voltage < 0.0:
    p_bidi.bidi_rx.pa.XPS2.heater_voltage = -rx_pa_voltage

  p_bidi.bidi_rx.mzi_1.XPS1.heater_voltage = 0.0
  p_bidi.bidi_rx.mzi_1.XPS2.heater_voltage = 0.0
  if rx_mzi_voltage > 0.0:
    p_bidi.bidi_rx.mzi_1.XPS1.heater_voltage = rx_mzi_voltage
  if rx_mzi_voltage < 0.0:
    p_bidi.bidi_rx.mzi_1.XPS2.heater_voltage = -rx_mzi_voltage
    
  p_bidi.recursive_update()

  output_state_forward = p_bidi.smatrix @ input_state_forward

  intensities = np.abs(output_state_forward)**2

  params.append([voltages[0],  voltages[1], voltages[2], voltages[3], intensities[0], intensities[1], intensities[2], intensities[3]])
  
  return -1 * intensities[3]

In [6]:
index = 1

p_bidi_dict = {}

for connector_return_loss, idd_mc in tqdm(product(connector_return_loss_range, range(mc_samples)), total=len(connector_return_loss_range)*mc_samples):
  p_bidi.fiber_1._rotation = (rotation_rx[idd_mc * no_fibers], rotation_ry[idd_mc * no_fibers], rotation_rz[idd_mc * no_fibers], rotation_delta[idd_mc * no_fibers])
  p_bidi.fiber_2._rotation = (rotation_rx[idd_mc * no_fibers + 1], rotation_ry[idd_mc * no_fibers + 1], rotation_rz[idd_mc * no_fibers + 1], rotation_delta[idd_mc * no_fibers + 1])
  p_bidi.fiber_3._rotation = (rotation_rx[idd_mc * no_fibers + 2], rotation_ry[idd_mc * no_fibers + 2], rotation_rz[idd_mc * no_fibers + 2], rotation_delta[idd_mc * no_fibers + 2])
  p_bidi.fiber_4._rotation = (rotation_rx[idd_mc * no_fibers + 3], rotation_ry[idd_mc * no_fibers + 3], rotation_rz[idd_mc * no_fibers + 3], rotation_delta[idd_mc * no_fibers + 3])
  p_bidi.fiber_5._rotation = (rotation_rx[idd_mc * no_fibers + 4], rotation_ry[idd_mc * no_fibers + 4], rotation_rz[idd_mc * no_fibers + 4], rotation_delta[idd_mc * no_fibers + 4])
  p_bidi.fiber_6._rotation = (rotation_rx[idd_mc * no_fibers + 5], rotation_ry[idd_mc * no_fibers + 5], rotation_rz[idd_mc * no_fibers + 5], rotation_delta[idd_mc * no_fibers + 5])
  p_bidi.fiber_7._rotation = (rotation_rx[idd_mc * no_fibers + 6], rotation_ry[idd_mc * no_fibers + 6], rotation_rz[idd_mc * no_fibers + 6], rotation_delta[idd_mc * no_fibers + 6])

  p_bidi.oc_1._return_loss = connector_return_loss
  p_bidi.oc_2._return_loss = connector_return_loss
  p_bidi.oc_3._return_loss = connector_return_loss
  p_bidi.oc_4._return_loss = connector_return_loss
  p_bidi.oc_5._return_loss = connector_return_loss
  p_bidi.oc_6._return_loss = connector_return_loss

  bounds = [(-2.7, 2.7), (-2.7, 2.7), (-2.7, 2.7), (-2.7, 2.7)]

  opts = {
    "maxiter": 1000,
    'xatol': 1e-6,
    'fatol': 1e-6,
  }

  params = []

  result = minimize(
    objective_function,
    ((0.0, 0.0, 0.0, 0.0)),
    bounds=bounds,
    method='Nelder-Mead',
    tol=1e-6,
    )
  
  bias_offset = np.linspace(-0.3, 0.3, num=7)

  input_state_forward = np.array([1, 0, 0, 0])
  input_state_reverse = np.array([0, 0, 1, 0])
  output_state_forward = np.array([])
  output_state_reverse = np.array([])

  for tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset in product(bias_offset, bias_offset, bias_offset, bias_offset):
    p_bidi.bidi_tx.pa.XPS1.heater_voltage = 0.0
    p_bidi.bidi_tx.pa.XPS2.heater_voltage = 0.0
    
    tx_pa_bias = params[-1][0] + tx_pa_bias_offset
    if tx_pa_bias > 0.0:
      p_bidi.bidi_tx.pa.XPS1.heater_voltage = tx_pa_bias
    if tx_pa_bias < 0.0:
      p_bidi.bidi_tx.pa.XPS2.heater_voltage = -tx_pa_bias

    p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = 0.0
    p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = 0.0

    tx_mzi_bias = params[-1][1] + tx_mzi_bias_offset
    if tx_mzi_bias > 0.0:
      p_bidi.bidi_tx.mzi_1.XPS1.heater_voltage = tx_mzi_bias
    if tx_mzi_bias < 0.0:
      p_bidi.bidi_tx.mzi_1.XPS2.heater_voltage = -tx_mzi_bias

    p_bidi.bidi_rx.pa.XPS1.heater_voltage = 0.0
    p_bidi.bidi_rx.pa.XPS2.heater_voltage = 0.0

    rx_pa_bias = params[-1][2] + rx_pa_bias_offset
    if rx_pa_bias > 0.0:
      p_bidi.bidi_rx.pa.XPS1.heater_voltage = rx_pa_bias
    if rx_pa_bias < 0.0:
      p_bidi.bidi_rx.pa.XPS2.heater_voltage = -rx_pa_bias

    p_bidi.bidi_rx.mzi_1.XPS1.heater_voltage = 0.0
    p_bidi.bidi_rx.mzi_1.XPS2.heater_voltage = 0.0

    rx_mzi_bias = params[-1][3] + rx_mzi_bias_offset
    if rx_mzi_bias > 0.0:
      p_bidi.bidi_rx.mzi_1.XPS1.heater_voltage = rx_mzi_bias
    if rx_mzi_bias < 0.0:
      p_bidi.bidi_rx.mzi_1.XPS2.heater_voltage = -rx_mzi_bias

    p_bidi.recursive_update()

    opf = np.array(p_bidi.smatrix @ input_state_forward).flatten()
    opr = np.array(p_bidi.smatrix @ input_state_reverse).flatten()

    # output_state_forward.append(p_bidi.smatrix @ input_state_forward)
    # output_state_reverse.append(p_bidi.smatrix @ input_state_reverse)

    output_state_forward = np.concatenate((output_state_forward, opf), axis=None)
    output_state_reverse = np.concatenate((output_state_reverse, opr), axis=None)

    if np.abs(output_state_forward[-1])**2== np.max(np.abs(output_state_forward)**2):
      tx_pa_bias_max = params[-1][0] + tx_mzi_bias
      tx_mzi_bias_max = params[-1][1] + tx_mzi_bias
      rx_pa_bias_max = params[-1][2] + rx_pa_bias
      rx_mzi_bias_max = params[-1][3] + rx_mzi_bias


  output_state_forward = np.abs(np.array(output_state_forward).reshape(4, len(bias_offset), len(bias_offset), len(bias_offset), len(bias_offset)))**2
  output_state_reverse = np.abs(np.array(output_state_reverse).reshape(4, len(bias_offset), len(bias_offset), len(bias_offset), len(bias_offset)))**2

  p_bidi_dict[index] = {
    "fiber_1": (rotation_rx[idd_mc * no_fibers], rotation_ry[idd_mc * no_fibers], rotation_rz[idd_mc * no_fibers], rotation_delta[idd_mc * no_fibers]),
    "fiber_2": (rotation_rx[idd_mc * no_fibers + 1], rotation_ry[idd_mc * no_fibers + 1], rotation_rz[idd_mc * no_fibers + 1], rotation_delta[idd_mc * no_fibers + 1]),
    "fiber_3": (rotation_rx[idd_mc * no_fibers + 2], rotation_ry[idd_mc * no_fibers + 2], rotation_rz[idd_mc * no_fibers + 2], rotation_delta[idd_mc * no_fibers + 2]),
    "fiber_4": (rotation_rx[idd_mc * no_fibers + 3], rotation_ry[idd_mc * no_fibers + 3], rotation_rz[idd_mc * no_fibers + 3], rotation_delta[idd_mc * no_fibers + 3]),
    "fiber_5": (rotation_rx[idd_mc * no_fibers + 4], rotation_ry[idd_mc * no_fibers + 4], rotation_rz[idd_mc * no_fibers + 4], rotation_delta[idd_mc * no_fibers + 4]),
    "fiber_6": (rotation_rx[idd_mc * no_fibers + 5], rotation_ry[idd_mc * no_fibers + 5], rotation_rz[idd_mc * no_fibers + 5], rotation_delta[idd_mc * no_fibers + 5]),
    "fiber_7": (rotation_rx[idd_mc * no_fibers + 6], rotation_ry[idd_mc * no_fibers + 6], rotation_rz[idd_mc * no_fibers + 6], rotation_delta[idd_mc * no_fibers + 6]),
    "optimization_bias": params[-1][0:4],
    "bias_offset": bias_offset,
    "peak_bias": (tx_pa_bias_max, tx_mzi_bias_max, rx_pa_bias_max, rx_mzi_bias_max),
    "connector_return_loss": connector_return_loss,
    "idd_mc": idd_mc,
    "output_state_forward": output_state_forward,
    "output_state_reverse": output_state_reverse,
  }
  index += 1

100%|██████████| 2004/2004 [1:31:03<00:00,  2.73s/it]


In [7]:
connector_return_loss = connector_return_loss_range
phase_shifts = phase_shifts.reshape(mc_samples, no_fibers)
fiber_rotation = np.zeros((no_fibers, mc_samples, len(connector_return_loss_range), 4))
optimization_bias = np.zeros((len(connector_return_loss_range), mc_samples, 4))
peak_bias = np.zeros((len(connector_return_loss_range), mc_samples, 4))
output_state_forward = np.zeros((len(connector_return_loss_range), mc_samples, 4, len(bias_offset), len(bias_offset), len(bias_offset), len(bias_offset)))
output_state_reverse = np.zeros((len(connector_return_loss_range), mc_samples, 4, len(bias_offset), len(bias_offset), len(bias_offset), len(bias_offset)))

index = 1
for idd_connector_return_loss in range(len(connector_return_loss_range)):
  for idd_mc in range(mc_samples):
    for idd_bias in range(4):
      optimization_bias[idd_connector_return_loss, idd_mc, idd_bias] = p_bidi_dict[index]["optimization_bias"][idd_bias]
      peak_bias[idd_connector_return_loss, idd_mc, idd_bias] = p_bidi_dict[index]["peak_bias"][idd_bias]
    output_state_forward[idd_connector_return_loss, idd_mc, :, :, :, :, :] = p_bidi_dict[index]["output_state_forward"]
    output_state_reverse[idd_connector_return_loss, idd_mc, :, :, :, :, :] = p_bidi_dict[index]["output_state_reverse"]
    index += 1

for idd_mc in range(mc_samples):
  for idd_fiber in range(no_fibers):
    for axis in range(4):
      fiber_rotation[idd_fiber, idd_mc, :, axis] = p_bidi_dict[idd_mc+1]["fiber_" + str(idd_fiber + 1)][axis]

In [8]:
p_bidi_dataset = xr.Dataset(
  data_vars=dict(
    phase_shifts=(["mc", "fiber"], phase_shifts),
    fiber_rotation=(["fiber", "mc", "connector_return_loss", "axis"], fiber_rotation),
    optimization_bias=(["connector_return_loss", "mc", "bias"], optimization_bias),
    peak_bias=(["connector_return_loss", "mc", "bias"], peak_bias),
    output_state_forward=(["connector_return_loss", "mc", "ports", "tx_pa_bias_offset", "tx_mzi_bias_offset", "rx_pa_bias_offset", "rx_mzi_bias_offset"], output_state_forward),
    output_state_reverse=(["connector_return_loss", "mc", "ports", "tx_pa_bias_offset", "tx_mzi_bias_offset", "rx_pa_bias_offset", "rx_mzi_bias_offset"], output_state_reverse),
  ),
  coords=dict(
    connector_return_loss=connector_return_loss,
    fiber_no = np.linspace(1, no_fibers, no_fibers),
    mc = np.linspace(1, mc_samples, mc_samples),
    axis = np.linspace(1, 4, 4),
    ports = np.linspace(1, 4, 4),
    bias_offset=p_bidi_dict[1]["bias_offset"],
    tx_mzi_bias_offset=p_bidi_dict[1]["bias_offset"],
    tx_pa_bias_offset=p_bidi_dict[1]["bias_offset"],
    rx_mzi_bias_offset=p_bidi_dict[1]["bias_offset"],
    rx_pa_bias_offset=p_bidi_dict[1]["bias_offset"],
  ),
  attrs=dict(
    description="Dataset for polarization controller MPI optimization",
    author="Ashwyn S",
    created="2024-01-24",
  ),
)

p_bidi_dataset.to_netcdf("p_bidi_mpi_dataset_20240124.nc")

In [9]:
p_bidi_dataset 

<xarray.Dataset>
Dimensions:                (mc: 501, fiber: 7, connector_return_loss: 4,
                            axis: 4, bias: 4, ports: 4, tx_pa_bias_offset: 7,
                            tx_mzi_bias_offset: 7, rx_pa_bias_offset: 7,
                            rx_mzi_bias_offset: 7, fiber_no: 7, bias_offset: 7)
Coordinates:
  * connector_return_loss  (connector_return_loss) int64 -30 -40 -50 -60
  * fiber_no               (fiber_no) float64 1.0 2.0 3.0 4.0 5.0 6.0 7.0
  * mc                     (mc) float64 1.0 2.0 3.0 4.0 ... 499.0 500.0 501.0
  * axis                   (axis) float64 1.0 2.0 3.0 4.0
  * ports                  (ports) float64 1.0 2.0 3.0 4.0
  * bias_offset            (bias_offset) float64 -0.3 -0.2 -0.1 0.0 0.1 0.2 0.3
  * tx_mzi_bias_offset     (tx_mzi_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * tx_pa_bias_offset      (tx_pa_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * rx_mzi_bias_offset     (rx_mzi_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
  * rx_pa_bias_offset      (rx_pa_bias_offset) float64 -0.3 -0.2 ... 0.2 0.3
Dimensions without coordinates: fiber, bias
Data variables:
    phase_shifts           (mc, fiber) float64 0.0 0.001792 ... 6.281 6.283
    fiber_rotation         (fiber, mc, connector_return_loss, axis) float64 5...
    optimization_bias      (connector_return_loss, mc, bias) float64 -0.3472 ...
    peak_bias              (connector_return_loss, mc, bias) float64 -2.952 ....
    output_state_forward   (connector_return_loss, mc, ports, tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset) float64 ...
    output_state_reverse   (connector_return_loss, mc, ports, tx_pa_bias_offset, tx_mzi_bias_offset, rx_pa_bias_offset, rx_mzi_bias_offset) float64 ...
Attributes:
    description:  Dataset for polarization controller MPI optimization
    author:       Ashwyn S
    created:      2024-01-24